<a href="https://colab.research.google.com/github/lisaong/hss/blob/master/05_physionet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Name:

Date:

```

# Sensor Data Analysis

Analyse "Bag of sensors" data from PhysioNet: https://physionet.org/physiobank/database/noneeg/

Under/Oversampling to align samples

Feature Extraction
- Statistical
- Spectral

Machine Learning
- Time-domain features
- Frequency-domain features

## Data Introduction

This database contains non-EEG physiological signals collected at Quality of Life Laboratory at University of Texas at Dallas, used to infer the neurological status (including physical stress, cognitive stress, emotional stress and relaxation) of 20 healthy subjects. The data was collected using non-invasive wrist worn biosensors and consists of electrodermal activity (EDA), temperature, acceleration, heart rate (HR), and arterial oxygen level (SpO2).

The experimental procedures involving human subjects described in this work were approved under UTD IRB # 12-29 by the Institutional Review Board at the University of Texas at Dallas, Richardson, Texas, USA. 

The dataset consists of 7 stages for 20 subjects: 

- First Relaxation: five minutes 
- Physical Stress: Stand for one minute, walk on a treadmill at one mile per hour for two minutes, then walk/jog on the treadmill at three miles per hour for two minutes. 
- Second Relaxation: five minutes
- Mini-emotional stress: 40 seconds (Note: This portion of the data, which was collected right before the cognitive stress task, is not explained in the paper.) During this 40 seconds, the “instructions” for the math portion of the cognitive stress (to count backwards by sevens, beginning with 2485, for three minutes) were read to the volunteer. 
- Cognitive Stress: Count backwards by sevens, beginning with 2485, for three minutes. Next, perform the Stroop test for two minutes. The volunteer was alerted to errors by a buzzer. The Stroop test consisted of reading the names of colors written in a different color ink, then saying what color the ink was. 
- Third Relaxation: five minutes. 
- Emotional Stress: The volunteer was told he/she would be shown a five minute clip from a horror movie in one minute. After the minute of anticipation, a clip from a zombie apocalypse movie, The Horde was shown. 
- Fourth Relaxation: five minutes. 

Note: We had not originally intended to count the reading of the instructions to count backwards as an emotional stress. After all, instructions were given for each of the tasks. Unlike the other instruction sets, however, this one created a stress response in many of the volunteers that was obvious to the test administrator as the test was being given.

## Load Data

Each subject has several datafiles:
- SubjectN_AccTempEDA.atr: annotation
- SubjectN_AccTempEDA.dat: data
- SubjectN_AccTempEDA.hea: header
- SubjectN_Sp02HR.dat: data
- SubjectN_Sp02HR.hea: header

These files are in the WFDB (WaveForm DataBase) format, and can be read using the `wfdb` python module.
(https://github.com/MIT-LCP/wfdb-python)

https://www.physionet.org/standards/npsg/Moody.pdf

In [0]:
# Download and extract the data files
!wget -q -O Subject10.zip https://github.com/lisaong/hss/raw/colab/data/physionet/Subject10.zip
!unzip -o Subject10.zip
!rm Subject10.zip

In [0]:
# Install the Waveform Database library to read the data files
!pip install wfdb

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.graphics.tsaplots import plot_acf
from scipy import fftpack, signal

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA

sns.set_style('whitegrid') # style

import wfdb

# render plots inline
%matplotlib inline

### Acc Temp EDA

Accelerometer, Temperature, Electrodermal Activity Data

In [0]:
# Read annotations
ann = wfdb.rdann('./Subject10_AccTempEDA', extension='atr', summarize_labels=True)
print(ann.__dict__)

In [0]:
# Read records
record_acc_temp_eda = wfdb.rdrecord('./Subject10_AccTempEDA')
print(record_acc_temp_eda.__dict__)

wfdb.plot_wfdb(record=record_acc_temp_eda, title='Subject10_AccTempEDA', annotation=ann, plot_sym=True, 
               time_units='seconds', figsize=(15, 10))

In [0]:
data_acc_temp_eda = record_acc_temp_eda.p_signal
data_acc_temp_eda.shape

### SpO2 HR

Arterial Oxygen Levels, Heartrate


In [0]:
record_spo2_hr = wfdb.rdrecord('./Subject10_SpO2HR')
print(record_spo2_hr.__dict__)

wfdb.plot_wfdb(record=record_spo2_hr, title='Subject10_SpO2HR', time_units='seconds', figsize=(15, 5))

In [0]:
data_spo2_hr = record_spo2_hr.p_signal
data_spo2_hr.shape

In [0]:
# number of acceleration, etc samples per second
record_acc_temp_eda.fs

In [0]:
# number of SpO2 and HR samples per second
record_spo2_hr.fs

## Aligning data of different frequencies

The two dataset frequencies (number of samples per second) are different.

To support processing both datasets at the same time, we need to match the frequencies.

This is a common situation when taking readings from different sensors or data sources.

Two strategies:
1. Upsampling the smaller frequency data. E.g: repeat samples or interpolate.
2. Downsampling the larger frequency data. E.g: replace with mean or median.

Which one to pick depends on requirements: whether you need to maintain precision of the higher frequency dataset.

Example: https://machinelearningmastery.com/resample-interpolate-time-series-data-python/

### Option 1: Upsampling SpO2 HR to 8 samples per second

This option duplicates the SpO2 HR data at the smaller sampling interval.

In [0]:
# create an index with 1 second timestamps, using the length of data_spo2_hr
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.period_range.html
#
# frequency strings: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

# for this dataset, the start date is just an arbitrary reference so that we can
# use resample()
per_second_index = pd.period_range(start='2019-01-01', periods=len(data_spo2_hr), freq='S')
per_second_index

In [0]:
# create a dataframe for SpO2 data using the above period index
df_spO2_hr = pd.DataFrame(data_spo2_hr, index=per_second_index, columns=record_spo2_hr.sig_name)
df_spO2_hr.head()

In [0]:
# upsample to match the frequency of the other data (8 times)

samples_per_sec = record_acc_temp_eda.fs // record_spo2_hr.fs # // converts float to int (ceiling)
samples_per_sec

In [0]:
# resample, then interpolate
# Note: whether interpolation makes sense depends on the sensor and use case
upsampled = df_spO2_hr.resample('125ms')

df_upsampled = upsampled.interpolate()
df_upsampled.head(10)

In [0]:
# inspect the resulting dataframe
# Note the number of rows has increased by a factor of 8
df_upsampled.info()

In [0]:
# Note: there are fewer values in the Acc dataframe, so we need to ignore the
# later entries from df_upsampled.

df_acc_temp_eda = pd.DataFrame(data_acc_temp_eda, columns=record_acc_temp_eda.sig_name)
df_acc_temp_eda.info()

In [0]:
# skip the first (18288-18239 = 49) entries
df_acc_temp_eda.index = df_upsampled.index[:len(df_acc_temp_eda)]
df_acc_temp_eda.info()

In [0]:
# concatenate the two dataframes, column-wise
# we now have Acc, Temp, EDA, SpO2, HR in 1 combined dataframe
df_option1 = pd.concat([df_acc_temp_eda, df_upsampled], axis=1).dropna()
df_option1.head()

In [0]:
# inspect the number of columns, rows, and datatype
df_option1.info()

In [0]:
# Plot the data
# https://stackoverflow.com/questions/48126330/python-int-too-large-to-convert-to-c-long-plotting-pandas-dates
df_option1.index = pd.to_datetime(df_option1.index.to_timestamp())

df_option1.plot(figsize=(15, 10))
ax = plt.gca()
ax.set_title('Upsampled Data')
plt.show()

### Option 2: Downsampling Acc Temp EDA to 1 sample per second

In the previous option, we duplicate SpO2 HR data to the smaller interval. 

<font color='red'>**Q1: What problems do you think this causes?**</font>

```
Enter your answer here:




```

An alternative approach is to sample the Acc Temp EDA data to match the larger interval of SpO2 HR.


In [0]:
# create an index with 125 millisecond timestamps, using the length of data_acc_temp_eda
#
# frequency strings: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

# for this dataset, the start date is just an arbitrary reference
per_125_ms_index = pd.period_range(start='2019-01-01', periods=len(data_acc_temp_eda), freq='125ms')
per_125_ms_index

In [0]:
# create a dataframe for Acc Temp EDA using the 125ms period index
df_acc_temp_eda2 = pd.DataFrame(data_acc_temp_eda, index=per_125_ms_index, columns=record_acc_temp_eda.sig_name)
df_acc_temp_eda2.head()

In [0]:
# downsample using median of Acc Temp EDA2
# using median is safer than using mean because median is less susceptible to outliers
# Can you think of the reason why median is less susceptible to outliers?

df_acc_temp_eda_downsampled = df_acc_temp_eda2.resample('S').median()
df_acc_temp_eda_downsampled.head(10)

In [0]:
# inspected the downsampled data
# Note the resulting number of rows has decreased by a factor of 8
df_acc_temp_eda_downsampled.info()

In [0]:
# create a dataframe so that we can concatenate
df_spo2_hr2 = pd.DataFrame(data_spo2_hr, columns=record_spo2_hr.sig_name, index=per_second_index)
df_spo2_hr2.info()

In [0]:
# concatenate the two dataframes, column-wise
df_option2 = pd.concat([df_acc_temp_eda_downsampled, df_spo2_hr2], axis=1).dropna()
df_option2.head()

In [0]:
# inspect the resulting dataframe of Acc Temp EDA SpO2 HR data
df_option2.info()

In [0]:
# plot the data
df_option2.index = pd.to_datetime(df_option2.index.to_timestamp())

df_option2.plot(figsize=(15, 10))
ax = plt.gca()
ax.set_title('Downsampled Data')
plt.show()

In [0]:
# Let's plot accelerometerX side-by-side

fig, ax = plt.subplots(figsize=(20, 10))

# zoom in to any 1 second time window
start_time = '2019-01-01 00:15'
end_time = '2019-01-01 00:16'

column = 'ax'

ax.plot(df_option1[(df_option1.index >= start_time) & (df_option1.index < end_time)][column],
        label='Upsampled (with interpolation)')
ax.plot(df_option2[(df_option2.index >= start_time) & (df_option2.index < end_time)][column],
        label='Downsampled (with median)')
ax.legend()
plt.show()

Generate plots for other columns, such as ay, temp, SpO2. 

<font color='red'>**Q2: What do you observe if you compare the upsampled and downsampled SpO2 or HR? Why?**</font>

```
Enter your answer here:




```

## Statistical Features

Basic statistical features you can get from time-series data, using Pandas and other python libraries:

- Mean, median, standard deviation
- Quantisation / discretisation
- Correlation
- Auto-correlation

In [0]:
df = df_option1.copy()

In [0]:
df.mean() # mean of each column

In [0]:
df.median() # median is less sensitive to outliers than mean

In [0]:
df.std() # standard deviation

In [0]:
df.max()

In [0]:
df.min()

### Discretise into quantiles

Another technique with Time Series data is to apply Discretisation.  Discretisation is useful when there is a lot of noise in the signal and you want to create models that work more generically (on "bins" of input values).

https://datascience.stackexchange.com/questions/19782/what-is-the-rationale-for-discretization-of-continuous-features-and-when-should

In [0]:
df.ax.values.ravel() # raw values

In [0]:
# discretise using qcut, 10 bins
# duplicates='drop' drops overlapping edges
df['ax_q10'] = pd.qcut(df.ax.values.ravel(), 10, duplicates='drop')
df.ax_q10.value_counts()

In [0]:
df['ax_q10']

In [0]:
df['ay_q10'] = pd.qcut(df.ay.values.ravel(), 10, duplicates='drop')
df['az_q10'] = pd.qcut(df.az.values.ravel(), 10, duplicates='drop')

In [0]:
df.ay_q10.value_counts()

In [0]:
df.az_q10.value_counts()

### Pair-plot

Pair plots are a combination of scatter plots and histograms. 

They are done for each pair of features (e.g. ax vs. ay)

https://seaborn.pydata.org/generated/seaborn.pairplot.html

In [0]:
sns.pairplot(df)
plt.show()

1. What do you notice about temp vs. EDA? **Are they positively or negatively correlated?**
2. Do you notice any **outliers** in the data? If yes, can you think of a simple way to remove outliers? (hint: something that uses the simple statistical features)


### Correlation

Correlations provide a metric to indicate whether two variables are strongly dependent.

https://www.statisticssolutions.com/correlation-pearson-kendall-spearman/

In [0]:
df.corr(method='pearson')

In [0]:
# a graphical way to viewing the correlation matrix
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df.corr(method='pearson'), annot=True, fmt='.2f')
plt.show()

## Spectal Features

Another way to extract features from time series data is to decompose the time series into its frequency components.

A common technique is to use Fast Fourier Transforms:

https://flothesof.github.io/Fourier-series-rectangle.html

https://ipython-books.github.io/101-analyzing-the-frequency-components-of-a-signal-with-a-fast-fourier-transform/

In [0]:
# Helper function to compute power spectral density
def power_spectral_density(x):
    psd = np.abs(x) ** 2
    return 20 * np.log10(psd / psd.max()) # decibels

In [0]:
# Compute fft of some signals
fft_ax = fftpack.fft(df.ax)
fft_hr = fftpack.fft(df.hr)
fft_eda = fftpack.fft(df.EDA)
# can add other signals as well

# Extract the power spectral density (squared magnitude of FFT, in decibels)
psd_ax = power_spectral_density(fft_ax)
psd_hr = power_spectral_density(fft_hr)
psd_eda = power_spectral_density(fft_eda)

sample_freq = fftpack.fftfreq(len(df.ax), d=1./samples_per_sec)

In [0]:
# plot the PSD of each FFT, focusing only on the positive frequencies
pos_mask = sample_freq > 0

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(15, 15))

ax1.plot(sample_freq[pos_mask], psd_ax[pos_mask])
ax1.set_title('ax')
ax1.set_ylabel('PSD (dB)')
ax1.set_xlabel('Frequency (Hz)')

ax2.plot(sample_freq[pos_mask], psd_hr[pos_mask])
ax2.set_title('hr')
ax2.set_ylabel('PSD (dB)')
ax2.set_xlabel('Frequency (Hz)')

ax3.plot(sample_freq[pos_mask], psd_eda[pos_mask])
ax3.set_title('EDA')
ax3.set_ylabel('PSD (dB)')
ax3.set_xlabel('Frequency (Hz)')

plt.show()

### Smoothing

The above FFTs are very noisy, because we are reading individual samples.

Let's try to apply a window technique to smooth out the samples.

An example is Hamming Window:
https://flothesof.github.io/FFT-window-properties-frequency-analysis.html

In [0]:
m = samples_per_sec * 5 # window size
w = signal.get_window('hamming', m)

fig, ax = plt.subplots()
ax.plot(w)
ax.set_xlabel('sample #')
ax.set_ylabel('amplitude')
plt.show()

The above is a hamming window of 5 seconds (8 samples per second).

Now, let's apply the windowing function to our signals, and then take the FFT.


In [0]:
# transform the signal into windows
smoothed_window = df.ax[:m] * w

fig, ax = plt.subplots()
ax.plot(df.ax[:m], label='before smoothing')
ax.plot(smoothed_window, label='after smoothing')
ax.legend()
ax.set_title('Hamming Window Illustration (5 seconds)')
fig.autofmt_xdate()

plt.show()

<font color='red'>**Q3: What is the difference between the 'after smoothing' plot compared to 'before smoothing'?**</font>

(Hint: look at the Hamming Window function's amplitude plot) 

```
Enter your answer here:



```

In [0]:
# Apply a rolling window transform onto the whole signal
df_smoothed = df.rolling(m, win_type='hamming').mean().dropna()

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(df.ax[m-1:], label='before smoothing')
ax.plot(df_smoothed.ax, label='after smoothing')
ax.set_title('Hamming Window for ax (whole series)')

fig.autofmt_xdate()
ax.legend()
plt.show()

In [0]:
# Recompute FFT on the smoothed signals

fft_ax = fftpack.fft(df_smoothed.ax)
fft_hr = fftpack.fft(df_smoothed.hr)
fft_eda = fftpack.fft(df_smoothed.EDA)

psd_ax = power_spectral_density(fft_ax)
psd_hr = power_spectral_density(fft_hr)
psd_eda = power_spectral_density(fft_eda)

sample_freq = fftpack.fftfreq(len(df_smoothed.ax), d=1./samples_per_sec)

pos_mask = sample_freq > 0

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(15, 15))

ax1.plot(sample_freq[pos_mask], psd_ax[pos_mask])
ax1.set_title('ax (smoothed)')
ax1.set_ylabel('PSD (dB)')
ax1.set_xlabel('Frequency (Hz)')

ax2.plot(sample_freq[pos_mask], psd_hr[pos_mask])
ax2.set_title('hr (smoothed)')
ax2.set_ylabel('PSD (dB)')
ax2.set_xlabel('Frequency (Hz)')

ax3.plot(sample_freq[pos_mask], psd_eda[pos_mask])
ax3.set_title('EDA (smoothed)')
ax3.set_ylabel('PSD (dB)')
ax3.set_xlabel('Frequency (Hz)')

plt.show()

## Machine Learning

In the next section, we can explore machine learning using first the time domain signal (with discretised features), and then later on using the frequency-domain features.

- Encode discretised features
- Add and encode labels
- Train an "emotional state" classification model
  - Classification using time-domain signal
  - Classification using frequency-domain features

### Encode discretised features

To add the discretised features, we can map the bins to categorical values.

In [0]:
le_ax_q10 = LabelEncoder()
df['ax_q10_enc'] = le_ax_q10.fit_transform(df['ax_q10'])

le_ay_q10 = LabelEncoder()
df['ay_q10_enc'] = le_ay_q10.fit_transform(df['ay_q10'])

le_az_q10 = LabelEncoder()
df['az_q10_enc'] = le_az_q10.fit_transform(df['az_q10'])

### Add and Encode Emotional State labels

Recall that the data was collected while the person is performing some relaxation / stressful activities.

The labels can be used to train a machine learning model to predict whether the sensor signals (HR, Acc, Temp) indicate stress or relaxation.

In [0]:
# find where the labels are stored in our dataset
ann.__dict__

In [0]:
# find which rows are the labels in the data series
ann.sample

In [0]:
ann.aux_note

In [0]:
# align the labels with the row offsets
# (row indices are 0-based)
start = ann.sample - 1
end = ann.sample[1:] - 1
end[-1] = -1 # last element

print('start indices', start)
print('end indices', end)

In [0]:
# create a new column with empty labels
df['label'] = ''

In [0]:
# mark the labels based on the start and end offsets
for s, e, label in zip(start, end, ann.aux_note):
    print(s, e, label) # s is inclusive, e is exclusive
    df.loc[s:e, 'label'] = label

In [0]:
df.tail()

In [0]:
# set the final row's label to the second-last row's label
df.loc[df.index[-1], 'label'] = df.loc[df.index[-2], 'label']

In [0]:
df.tail()

In [0]:
# Label Encode

le = LabelEncoder()
df['label_enc'] = le.fit_transform(df['label'])
df.head()

In [0]:
# Check correlation after encoding
df.corr()['label_enc']

<font color='red'>**Q4: Based on the correlations, what top 5 features will you pick, and why?**</font>

```
Enter your answer here:




```

### Shuffle and Train-test split

In machine learning, the customary practice is to set apart some data for independent testing.

The objective is to ensure that the resulting model is **generalisable**, meaning that it work well for data that is not seen before. This also detects a problem known as **overfitting**, where the learnt model is too specialised for the data it has seen. Typically this happens for more complicated models where you can overtrain.

In [0]:
df_train, df_test = train_test_split(df, random_state=42) # randomise features the same way each time (repeatability)

### Feature Selection

Previously, we used correlation as a way to decide which features may work "better" for predicting the target.

This test can be automated using Select K-best features, which computes correlation of each candidate feature with the target.

In [0]:
# exclude the label, encoded label, and the quantised intervals
candidate_features = df.columns.drop(['label', 'label_enc', 'ax_q10', 'ay_q10', 'az_q10'])
candidate_features

In [0]:
# we arbitrarily pick k=5 as a starting point. This can be tuned later on.
kbest = SelectKBest(k=5, score_func=f_classif)
kbest.fit(df_train.loc[:, candidate_features], df_train.label_enc)

kbest.scores_

In [0]:
# best 5 features (note that this is not in k-score ordering, but in 
# the original order of the columns)
candidate_features[kbest.get_support()]

Compare the best 5 features from kbest against actual correlation from the cell below:

<font color='red'>**Q5: How do the best 5 features from kbest relate to the Actual correlation values in the next cell?**</font>

 ```
Enter your answer here:



 ```

In [0]:
# Actual correlation
df.corr()['label_enc']

In [0]:
# Perform feature selection
X_train = kbest.transform(df_train.loc[:, candidate_features])
X_test = kbest.transform(df_test.loc[:, candidate_features])
y_train = df_train.label_enc
y_test = df_test.label_enc

### Bayes classifier

Uses Bayes Theorem and counting to compute conditional probability of each type of event.

The probabilities are modeled using the gaussian probability distribution function.

This is a good "starter" model because it is simple and interpretable. The interpretability comes from the way the predictions are generated - based on probability of feature values for a given label.

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

In [0]:
nb = GaussianNB()
nb.fit(X_train, y_train)
print('Accuracy', nb.score(X_test, y_test))

In [0]:
pred_nb = nb.predict(X_test)
print(classification_report(y_test, pred_nb, target_names=le.classes_))

fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(y_test, pred_nb), annot=True, fmt='d',
            xticklabels=le.classes_, yticklabels=le.classes_, ax=ax)
ax.set_xlabel('Prediction')
ax.set_ylabel('Truth')
plt.show()

### Random Forest Classifier

A Naive Bayes model doesn't really support tuning.  Let's try an ensemble algorithm such as Random Forest.

This creates a number of decision tree classifiers on subsets of the dataset, and averages the results.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [0]:
# max_depth and n_estimators limit overfitting
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

rf.fit(X_train, y_train)
rf.score(X_test, y_test)

print('Accuracy', rf.score(X_test, y_test))

In [0]:
pred_rf = rf.predict(X_test)
print(classification_report(y_test, pred_rf, target_names=le.classes_))

fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(y_test, pred_rf), annot=True, fmt='d',
            xticklabels=le.classes_, yticklabels=le.classes_, ax=ax)
ax.set_xlabel('Prediction')
ax.set_ylabel('Truth')
plt.show()

### Tuning

We can perform a grid search both kBest and on combinations of RandomForest hyperparameters to find a better model.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [0]:
# Earlier we used k=5 for kBest, now we can actually tune it while
# training the model
pipeline = Pipeline(steps=[
  ('kbest', SelectKBest()),
  ('rf', RandomForestClassifier())
])

# setup the parameter grid to search
# the format is: stepname__parameter
param_grid = {
    'kbest__k' : [3, 5, 7],
    'rf__max_depth': [5, 7, 9, 11, 13],
    'rf__n_estimators': [75, 100, 125, 150, 175, 200]
}

X_train_before_kbest = df_train.loc[:, candidate_features]
X_test_before_kbest = df_test.loc[:, candidate_features]

# Run 4 jobs concurrently, using cross validation split of 3
gs = GridSearchCV(pipeline, param_grid, n_jobs=4, cv=3, verbose=True, 
                  return_train_score=True)
gs.fit(X_train_before_kbest, y_train)
print('Accuracy', gs.score(X_test_before_kbest, y_test))
print('Best combination', gs.best_params_)

In [0]:
# get the full results
pd.DataFrame(gs.cv_results_)

In [0]:
pred_gs = gs.predict(X_test_before_kbest)
print(classification_report(y_test, pred_gs, target_names=le.classes_))

fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(y_test, pred_gs), annot=True, fmt='d',
            xticklabels=le.classes_, yticklabels=le.classes_, ax=ax)
ax.set_xlabel('Prediction')
ax.set_ylabel('Truth')
plt.show()

<font color='red'>**Q6: What are the best k, max_depth, and n_estimators hyperparameters?**</font>

(Hint: use gs.best_params_)

```
Enter your answer here:




```

You can read up on max_depth and n_estimators here: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

You can read up on GridSearchCV here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

## Train using frequency domain features

If time domain features aren't performing well (not really the case here), we can also try using the FFT to fit the model.

1. Take rolling windows of the signal
2. Compute spectral features based on the FFT
3. Use both original and spectral features in our model

To learn, we'll be computing the spectral features "by hand", but you can alternatively consider a package like tsfresh that calculates and evaluates the features automatically:
* https://tsfresh.readthedocs.io/en/latest/text/introduction.html
* https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.fft_aggregated

(Note, however, tsfresh is even more of a "blackbox" library)


In [0]:
def compute_fft(df, window_size=1, samples_per_sec=8, smoothing=False):
    if smoothing:
        df_ = df.rolling(m, win_type='hamming').mean().dropna()
    else:
        df_ = df
    
    # fftpack.fftfreq returns the discrete fourier transform sample frequencies
    # len(df_) is the window length
    # sample spacing = 1./samples_per_sec
    sample_freq_ = fftpack.fftfreq(len(df_), d=1./samples_per_sec)

    # only take the positive frequencies (negative frequencies don't make sense)
    pos_mask_ = sample_freq_ > 0

    # wrap the result into a dataframe    
    result_df_ = pd.DataFrame(index = sample_freq_[pos_mask_])
    
    # Fill in fft value for each column
    # FFT needs to be computed per column, hence we are using a loop
    for c in df_.columns:
        if df_[c].dtype == float:
            fft_ = np.abs(fftpack.fft(df_[c])) # take the magnitude
            result_df_[c + '_fft'] = fft_[pos_mask_]
    
    result_df_.index.name = 'sampling_frequency'

    return result_df_

In [0]:
# uncomment this to get the help information
# fftpack.fftfreq?

Since FFT is in the frequency domain, we will need to break up the time signal into time-windows.

Let's say we have time windows of 10 seconds, and we extract FFT characteristics from each window.

Later on, we will be sliding the time-window across the entire duration. 

In [0]:
time_window_sec = 10
start_index = 0
end_index = time_window_sec * samples_per_sec # 10 sec * 8 samples/sec

df_time_window = df.iloc[start_index:end_index]
fft_time_window = compute_fft(df_time_window)

fig, ax = plt.subplots()
fft_time_window.plot(ax=ax) # plotting this way adds the correct labels

ax.set_title(f'FFT: using {time_window_sec} second window')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('FFT magnitude')
plt.show()

In [0]:
fft_time_window

In [0]:
# number of rows indicate the number of sampling frequencies
# each entry is the absolute magnitude of the FFT at that frequency
fft_time_window.info()

#### Spectral Centroid

This corresponds to the balancing point or the center of mass of the spectral power distribution.
This value is calculated as the weighted mean of the frequency components present in the signal:

$$\frac{\sum{F(n) \times n}}{\sum F(n)}$$

In [0]:
# Spectral centroid is the weighted mean of the frequency components
# fft_time_window.index are the frequencies
spectral_centroid = np.dot(fft_time_window.index, fft_time_window) / np.sum(fft_time_window)
spectral_centroid

#### Spectral Energy

The equivalent to the energy of the signal as discussed before in
the frequency domain is the spectral energy. This value is computed as the sum of
the squares of the magnitude of the frequency content:

$$\sum{F(n)^2}$$

In [0]:
# Spectral energy
np.sum(fft_time_window ** 2) # sum of squares

#### Spectral Entropy

This refers to the entropy of the signal in the frequency domain. The frequency content of the signal is normalized before the entropy computation.

$$-\sum{\hat{F}(n) \times log(\hat{F}(n))} $$

In [0]:
# normalise the FFT by dividing by sum
normalized_fft_time_window = fft_time_window / np.sum(fft_time_window)

# compute entropy
-np.sum(normalized_fft_time_window * np.log(normalized_fft_time_window))

### Computing Spectal Features

We'll wrap the computation into a function and then call that function using the pd.DataFrame.rolling() method.


In [0]:
def compute_fft_series(df, window_size=1, samples_per_sec=8, smoothing=False):
    if smoothing:
        df_ = df.rolling(m, win_type='hamming').mean().dropna()
    else:
        df_ = df
    
    sample_freq_ = fftpack.fftfreq(len(df_), d=1./samples_per_sec)
    pos_mask_ = sample_freq_ > 0
    
    fft_ = np.abs(fftpack.fft(df_))
    result_df_ = pd.Series(index=sample_freq_[pos_mask_], data=fft_[pos_mask_])
    result_df_.index.name = 'sampling_frequency'

    return result_df_    

def spectral_centroid(df):
    fft_ = compute_fft_series(df, window_size=1, samples_per_sec=samples_per_sec, smoothing=False)
    return np.dot(fft_.index, fft_) / np.sum(fft_)

def spectral_energy(df):
    fft_ = compute_fft_series(df, window_size=1, samples_per_sec=samples_per_sec, smoothing=False)
    return np.sum(fft_ ** 2)

def spectral_entropy(df):
    fft_ = compute_fft_series(df, window_size=1, samples_per_sec=samples_per_sec, smoothing=False)
    normalized_fft_ = fft_ / np.sum(fft_)
    return -np.sum(normalized_fft_ * np.log(normalized_fft_))

# Example usage:
window_size_secs = 10

# raw = False applies to a pandas Series
# df['ay'].rolling(window_size_secs * samples_per_sec).apply(spectral_centroid, raw=False).dropna().head(10)
# df['ay'].rolling(window_size_secs * samples_per_sec).apply(spectral_energy, raw=False).dropna().head(10)
df['ay'].rolling(window_size_secs * samples_per_sec).apply(spectral_entropy, raw=False).dropna().head(10)

In [0]:
# columns to generate spectral features from
# In general, you want to use the raw columns instead of the discretised columns, because 
# there is a loss of information from the discretised columns.
float_columns = ['ax', 'ay', 'az', 'temp', 'EDA', 'SpO2', 'hr']

# This can take a while to run... because we do an FFT per column, 3 times
# While groupby supports multiple functions, groupby does not do rolling windows, 
# so we have to call rolling 3 times
for f in float_columns:
    df[f + '_sc'] = df[f].rolling(window_size_secs * samples_per_sec).apply(spectral_centroid, raw=False)
    df[f + '_sen'] = df[f].rolling(window_size_secs * samples_per_sec).apply(spectral_energy, raw=False)
    df[f + '_se'] = df[f].rolling(window_size_secs * samples_per_sec).apply(spectral_entropy, raw=False)

df.dropna(inplace=True)
df.head(10)

In [0]:
# move the label column to the front so that it is easy to see
new_cols = ['label', 'label_enc'] + df.columns.drop(['label', 'label_enc']).tolist()

df = df[new_cols]
df.head()

In [0]:
df.corr()['label_enc']

In [0]:
# filter for higher correlations
mask = abs(df.corr()['label_enc']) > 0.2
df.corr()['label_enc'][mask]

### Shuffle and Train-test split

With the extra spectral features, we will see if we can improve the simple (Bayes) model.

As stated before, the Bayes model is more interpretable because it is probability-based.

In [0]:
# Train test split
df_train_sp, df_test_sp = train_test_split(df, random_state=42)

In [0]:
# exclude the label, encoded label, and the quantised intervals
candidate_features_sp = df.columns.drop(['label', 'label_enc', 'ax_q10', 'ay_q10', 'az_q10'])
candidate_features_sp

### Bayes classifier (revisited)

<font color='red'>**Q7: Apply the new features train a new Bayes Classifier. Use k=8 for SelectKBest. What is the new accuracy score and classification report?**</font>

```
Enter your answer here:




```


In [0]:
# 1. apply SelectKBest, with k=8 (giving some room as we have more features)
kbest_sp = SelectKBest(k=8)
kbest_sp.fit(df_train_sp.loc[:, candidate_features_sp], df_train_sp.label_enc)

# Perform feature selection
X_train_sp = kbest_sp.transform(df_train_sp.loc[:, candidate_features_sp])
y_train_sp = df_train_sp.label_enc
X_test_sp = kbest_sp.transform(df_test_sp.loc[:, candidate_features_sp])
y_test_sp = df_test_sp.label_enc

# 2. apply GaussianNB on the selected 8 features
nb_sp = GaussianNB()
#
# Enter your code below
#



# 3. compute the accuracy score on the test set
#
# Enter your code below
#



# 4. compute the classification report and confusion matrix on the test set
#
# Enter your code below
#








## Decision Boundaries

(You can never be done with analysis).

Let's see if we can compare the decision boundaries, to get a better sense of how the features are spread out.

1. Perform 2-D PCA so that we can visualise all the features in a 2-D plot
2. Create a mesh grid, colour regions based on predictions from the classifier
3. Repeat step 2 for each classifier

In [0]:
# PCA presumes zero centered mean, so we scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_sp)
X_test_scaled = scaler.transform(X_test_sp)

pca = PCA(n_components=2)

Z_train_pca = pca.fit_transform(X_train_scaled)
Z_test_pca = pca.transform(X_test_scaled)

# Train and test sets are shuffled by train_test_split before the split,
# so we need to concatenate instead of using X and y.
# Also, we want to use the scaled version of X
Z_pca = np.concatenate((Z_train_pca, Z_test_pca), axis=0)
y_all = np.concatenate((y_train_sp, y_test_sp), axis=0)

### Plot data points

In [0]:
fig, ax = plt.subplots(figsize=(15, 10))

for i in range(0, len(le.classes_)):
    # y_train == i filters a subset of rows where y_train is i    
    # The ..., 0] refers to the first principal component
    # The ..., 1] refers to the second principal component
    ax.scatter(Z_pca[y_all == i, 0], Z_pca[y_all == i, 1], label=le.classes_[i])
ax.legend()

ax.set_title('PCA plot of dataset including Spectral Features')
ax.set_xlabel('Principal component 1')
ax.set_ylabel('Principal component 2')
plt.show()

### Plot decision boundaries

In [0]:
def plot_decision_boundaries(ax, title, clf, data, target, step_size=.02):
    """Plots the decision boundaries for a fitted classifier model
    Args:
        ax: subplot axis
        title: subplot title
        clf: a fitted sklearn classification model
        data: 2-dimensional input data
        target: the target truth values (y)
        step_size: Step size of the mesh. Decrease to increase the quality of the plot.
        
    Globals:
        pca: fitted PCA transformer for projecting 2-dimensional to original X dimension
        scaler: fitted StandardScaler for unscaling the X back to original scale
        le: fitted LabelEncoder for mapping to class labels

    Based on: http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html
    """
    # Generate a mesh of many points, this will become regions when coloured
    x_min_, x_max_ = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min_, y_max_ = data[:, 1].min() - 1, data[:, 1].max() + 1
    xx_, yy_ = np.meshgrid(np.arange(x_min_, x_max_, step_size), np.arange(y_min_, y_max_, step_size))

    # Make 2 columns (principal component 1, principal component 2)
    zz_ = np.c_[xx_.ravel(), yy_.ravel()]
    
    # We need to inverse both PCA and scaling because clf is trained on unscaled and un-PCA'ed data
    z_ = pca.inverse_transform(zz_) # inverse-PCA
    z_ = scaler.inverse_transform(z_) # unscale
    
    # Obtain labels for each point in mesh using the trained model
    pred_ = clf.predict(z_)

    # Put the result into a color plot
    pred_ = pred_.reshape(xx_.shape)

    # Plot the decision boundary. Assign a colour based on prediction
    ax.imshow(pred_, interpolation='nearest',
              extent=(xx_.min(), xx_.max(), yy_.min(), yy_.max()),
              cmap=plt.cm.Pastel2,
              aspect='auto', origin='lower')

    # Plot the points. Assign a colour based on truth
    # Made the points smaller and more transparent so that the boundaries are still visible.
    for i in range(0, len(le.classes_)):
        ax.scatter(data[target == i, 0], data[target == i, 1], label=le.classes_[i],
                   alpha=.6)

    ax.set(title=title, xlim=(x_min_, x_max_), ylim=(y_min_, y_max_),
           xlabel='Principal component 1', ylabel='Principal component 2')
    ax.legend()

In [0]:
# Just plot the test set to speed things up
fig, ax1 = plt.subplots(figsize=(15, 10))
plot_decision_boundaries(ax1, 'Naive Bayes Classifier Decision Boundaries', nb_sp, Z_test_pca, y_test_sp)
plt.show()

Naive Bayes is a simple model that produces gaussian-shaped decision boundaries. 

Note: The boundaries are also projected in 2-D, so it looks like there are more mistakes here, but each actual boundary is N dimension.